<a href="https://colab.research.google.com/github/GlenM42/PCA/blob/main/DataMining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem 1: PCA

In [1]:
import numpy as np

# This is going to be our function to perform a PCA:
def pca(data, n_components=None):
  """
  :param data: numpy array of shape (n_samples, n_features)
  :param n_components: number of principal components to return, optional
  :return: transformed data, eigenvalues, eigenvectors
  """

  # Step 1: Center the data
  mean = np.mean(data, axis=0)
  centered_data = data - mean

  # Step 2: Compute the covariance matrix
  cov_matrix = np.cov(centered_data, rowvar=False)

  # Step 3: Solve the char polynomial
  eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

  # Step 3.5: Find the char polynomial
  char_poly = np.poly(eigenvalues)

  # Step 4: Sort eigenvalues and eigenvectors in descending order of eigenvalues
  sorted_ind = np.argsort(eigenvalues)[::-1]
  sorted_eigenvalues = eigenvalues[sorted_ind]
  sorted_eigenvectors = eigenvectors[:, sorted_ind]

  # Step 5: Select the top n_comp if specified
  if n_components is not None:
    sorted_eigenvectors = sorted_eigenvectors[:, :n_components]

  # Step 6: Calculate the explained variance
  total_var = np.sum(sorted_eigenvalues)
  explained_var = (sorted_eigenvalues / total_var) * 100

  # Step 7: Transform the data using the selected eigenvectors
  transformed_data = np.dot(centered_data, sorted_eigenvectors)

  return {
      "mean": mean,
      'centered_data': centered_data,
      'char_poly': char_poly,
      'cov_matrix': cov_matrix,
      'eigenvalues': sorted_eigenvalues,
      'eigenvectors': sorted_eigenvectors,
      'explained_variance': explained_var,
      'transformed_data': transformed_data
  }

With this function defined, let's use it on our array:

In [2]:
results = pca(np.array([[5, 3, 95],
                        [6, 4, 98],
                        [7, 5, 100],
                        [8, 6, 102],
                        [9, 7, 104]]))

# Output the results
for key, value in results.items():
  print(f"{key.capitalize()}: \n{value}\n")

Mean: 
[ 7.   5.  99.8]

Centered_data: 
[[-2.  -2.  -4.8]
 [-1.  -1.  -1.8]
 [ 0.   0.   0.2]
 [ 1.   1.   2.2]
 [ 2.   2.   4.2]]

Char_poly: 
[ 1.00000000e+00 -1.72000000e+01  5.00000000e-01 -3.03153062e-16]

Cov_matrix: 
[[ 2.5  2.5  5.5]
 [ 2.5  2.5  5.5]
 [ 5.5  5.5 12.2]]

Eigenvalues: 
[1.71708809e+01 2.91190651e-02 6.06306124e-16]

Eigenvectors: 
[[-3.80779846e-01 -5.95824394e-01 -7.07106781e-01]
 [-3.80779846e-01 -5.95824394e-01  7.07106781e-01]
 [-8.42622939e-01  5.38504022e-01 -9.30532457e-15]]

Explained_variance: 
[9.98307031e+01 1.69296890e-01 3.52503560e-15]

Transformed_data: 
[[ 5.56770949e+00 -2.01521729e-01  3.58730602e-15]
 [ 2.27828098e+00  2.22341549e-01 -3.78954173e-15]
 [-1.68524588e-01  1.07700804e-01 -1.86106491e-15]
 [-2.61533016e+00 -6.93993973e-03  6.74119041e-17]
 [-5.06213573e+00 -1.21580684e-01  1.99588872e-15]]



# Problem 2: Binary Split

In [4]:
import numpy as np
from collections import Counter

# Function to calculate Gini index
def gini(y):
    counts = Counter(y)
    impurity = 1 - sum((count / len(y))**2 for count in counts.values())
    return impurity

# Function to calculate Entropy
def entropy(y):
    counts = Counter(y)
    impurity = -sum((count / len(y)) * np.log2(count / len(y)) for count in counts.values() if count != 0)
    return impurity

# Function to perform the binary split and return the best split point
def best_split(X, y, feature_labels, method="gini", attribute_type="continuous"):
    if method == "gini":
        impurity_func = gini
    elif method == "entropy":
        impurity_func = entropy
    else:
        raise ValueError("Method must be 'gini' or 'entropy'")

    best_split_point = None
    best_impurity = float('inf')
    best_left = None
    best_right = None
    best_feature = None

    for col in range(X.shape[1]):
        values = X[:, col]
        sorted_indices = np.argsort(values)
        sorted_values, sorted_labels = values[sorted_indices], y[sorted_indices]
        for i in range(1, len(sorted_values)):
            split_point = (sorted_values[i - 1] + sorted_values[i]) / 2
            left_labels = sorted_labels[:i]
            right_labels = sorted_labels[i:]
            left_impurity = impurity_func(left_labels)
            right_impurity = impurity_func(right_labels)
            impurity = (len(left_labels) / len(y)) * left_impurity + (len(right_labels) / len(y)) * right_impurity

            if impurity < best_impurity:
                best_impurity = impurity
                best_split_point = split_point
                best_left = (X[sorted_indices[:i], :], y[sorted_indices[:i]])
                best_right = (X[sorted_indices[i:], :], y[sorted_indices[i:]])
                best_feature = feature_labels[col]  # Track the best feature

    return best_split_point, best_left, best_right, best_impurity, best_feature

# Main function d_tree to perform the binary split and output important metrics
def d_tree(X, y, feature_labels, method="gini", attribute_type="continuous"):
    best_split_point, best_left, best_right, best_impurity, best_feature = best_split(X, y, feature_labels, method, attribute_type)

    print(f"Best Feature to Split On: {best_feature}")
    print(f"Best Split Point: {best_split_point}")
    print(f"Best Left Group Shape: {best_left[0].shape}")
    print(f"Best Right Group Shape: {best_right[0].shape}")
    print(f"Best Impurity (Gini/Entropy): {best_impurity}")

    print("\nLeft Group Labels Distribution:")
    print(Counter(best_left[1]))

    print("\nRight Group Labels Distribution:")
    print(Counter(best_right[1]))

With the functions defined, we can do the calculations on our data:

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Example dataset
data = [
    [1, 'M', 'Family', 'Small', 'C0'],
    [2, 'M', 'Sports', 'Medium', 'C0'],
    [3, 'M', 'Sports', 'Medium', 'C0'],
    [4, 'M', 'Sports', 'Large', 'C0'],
    [5, 'M', 'Sports', 'Extra Large', 'C0'],
    [6, 'M', 'Sports', 'Extra Large', 'C0'],
    [7, 'F', 'Sports', 'Small', 'C0'],
    [8, 'F', 'Sports', 'Small', 'C0'],
    [9, 'F', 'Sports', 'Medium', 'C0'],
    [10, 'M', 'Luxury', 'Large', 'C0'],
    [11, 'M', 'Family', 'Large', 'C1'],
    [12, 'M', 'Family', 'Extra Large', 'C1'],
    [13, 'M', 'Family', 'Medium', 'C1'],
    [14, 'M', 'Family', 'Extra Large', 'C1'],
    [15, 'F', 'Luxury', 'Small', 'C1'],
    [16, 'F', 'Luxury', 'Medium', 'C1'],
    [17, 'F', 'Luxury', 'Medium', 'C1'],
    [18, 'F', 'Luxury', 'Medium', 'C1'],
    [19, 'F', 'Luxury', 'Medium', 'C1'],
    [20, 'F', 'Luxury', 'Large', 'C1']
]
feature_labels = ['Gender', 'Car Type', 'Shirt Size']

# Convert the data into a NumPy array and split into features (X) and labels (y)
data = np.array(data)
X_raw = data[:, 1:4]  # Features
y = data[:, -1]       # Labels

# One-hot encoding for categorical features (Gender, Car Type, Shirt Size)
encoder = OneHotEncoder(sparse_output=False)
X_encoded = encoder.fit_transform(X_raw)
encoded_feature_labels = encoder.get_feature_names_out(feature_labels)

# Perform the decision tree binary split using Gini for the encoded categorical data
d_tree(
    X=X_encoded,
    y=y,
    feature_labels=encoded_feature_labels,
    method="entropy",
    attribute_type="categorical")

Best Feature to Split On: Car Type_Luxury
Best Split Point: 0.0
Best Left Group Shape: (9, 9)
Best Right Group Shape: (11, 9)
Best Impurity (Gini/Entropy): 0.2417233428068324

Left Group Labels Distribution:
Counter({'C0': 9})

Right Group Labels Distribution:
Counter({'C1': 10, 'C0': 1})
